In [92]:
# import der benötigten Bibliotheken

########################################################################################################################
# importieren notwendiger bibliotheken
import pandas as pd # Datenverarbeitung     
import seaborn as sns # Visualisierung
import numpy as np  # Mathe
import matplotlib.pyplot as plt #   Visualisierung


# Imports Sklearn
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Imports Deep Learning Pytorch
import torch
import torch.nn as nn

# Ausblenden von Warnungen um prints besser zu lesen
import warnings
warnings.filterwarnings('ignore')


# eigene Bibliothek für Plots
from molib import * 
plot = Plot(storage_path='../1_plots')

# Livedaten vom Campus

In [93]:
'''########################################################################################################################
# eigene Bibliothek für Datenbankzugriff
from library_database import influxDB
from datetime import datetime, timedelta
from time import sleep, time
import pandas as pd
influxDB_object = influxDB()


# datetime mit expliziten Datum
yesterday = datetime(2023, 12, 20)
#yesterday = datetime.now() - timedelta(15)
pv_power2 = influxDB_object.read_daily_frame(day=yesterday, sensor='616_PowerEl_PV2AC', bucket='Raw')
#pv_power2.index = pd.to_datetime(pv_power2.index)
pv_power2= pv_power2.resample('60min').mean()'''

"########################################################################################################################\n# eigene Bibliothek für Datenbankzugriff\nfrom library_database import influxDB\nfrom datetime import datetime, timedelta\nfrom time import sleep, time\nimport pandas as pd\ninfluxDB_object = influxDB()\n\n\n# datetime mit expliziten Datum\nyesterday = datetime(2023, 12, 20)\n#yesterday = datetime.now() - timedelta(15)\npv_power2 = influxDB_object.read_daily_frame(day=yesterday, sensor='616_PowerEl_PV2AC', bucket='Raw')\n#pv_power2.index = pd.to_datetime(pv_power2.index)\npv_power2= pv_power2.resample('60min').mean()"

# historische Wetterdaten und PV-Daten

In [94]:
#####################################
if 0:
    pv_power1 = pd.read_csv("../0_data/raw/611_PowerEl_PV1AC.csv", index_col=0, parse_dates=True)
    pv_power2 = pd.read_csv("../0_data/raw/616_PowerEl_PV2AC.csv", index_col=0, parse_dates=True)
    # ebh fehlt
    weather_data = pd.read_csv('../0_data/raw/solcast_PT5M.csv', index_col='period_end', parse_dates=True, usecols=['ghi','dni','dhi','air_temp','zenith','azimuth','cloud_opacity','period_end'])
    # period end zu perod begin verschieben
    weather_data.index = weather_data.index.shift(-1, freq='5T')

    # merge von pv_power1 und pv_power2
    pv_power = pd.merge(pv_power1, pv_power2, left_index=True, right_index=True)
    # merge von pv_power und weather_data
    pv_power = pd.merge(pv_power, weather_data, left_index=True, right_index=True)

    historic_data = pv_power.copy()
    # Umbennen der Pv leistungs spalten
    historic_data.rename(columns={'611_PowerEl_PV1AC':'pv_power1', '616_PowerEl_PV2AC':'pv_power2'}, inplace=True)
    historic_data.to_csv('../0_data/raw/merged_raw_data.csv')
else:
    historic_data = pd.read_csv('../0_data/raw/merged_raw_data.csv', index_col=0, parse_dates=True)

# Interval anpassen
historic_data = historic_data.resample('60min').mean()

historic_data

pv_power1  pv_power2  air_temp    azimuth  \
2020-01-01 00:00:00+00:00        0.0        0.0 -1.000000 -33.333333   
2020-01-01 01:00:00+00:00        0.0        0.0 -1.000000 -55.500000   
2020-01-01 02:00:00+00:00        0.0        0.0 -1.500000 -72.000000   
2020-01-01 03:00:00+00:00        0.0        0.0 -2.000000 -85.333333   
2020-01-01 04:00:00+00:00        0.0        0.0 -2.000000 -96.333333   
...                              ...        ...       ...        ...   
2024-01-19 19:00:00+00:00        0.0        0.0 -5.833333  80.833333   
2024-01-19 20:00:00+00:00        0.0        0.0 -6.000000  67.166667   
2024-01-19 21:00:00+00:00        0.0        0.0 -6.000000  50.000000   
2024-01-19 22:00:00+00:00        0.0        0.0 -6.000000  27.333333   
2024-01-19 23:00:00+00:00        0.0        0.0 -6.000000  -0.666667   

                           cloud_opacity  dhi  dni  ghi      zenith  
2020-01-01 00:00:00+00:00       0.000000  0.0  0.0  0.0  150.416667  
2020-01-01 01:00:00+00:00       0.000000  0.0  0.0  0.0  143.666667  
2020-01-01 02:00:00+00:00       0.000000  0.0  0.0  0.0  134.916667  
2020-01-01 03:00:00+00:00       0.000000  0.0  0.0  0.0  125.333333  
2020-01-01 04:00:00+00:00       0.000000  0.0  0.0  0.0  115.583333  
...                                  ...  ...  ...  ...         ...  
2024-01-19 19:00:00+00:00      83.950000  0.0  0.0  0.0  125.000000  
2024-01-19 20:00:00+00:00      83.783333  0.0  0.0  0.0  134.500000  
2024-01-19 21:00:00+00:00      74.283333  0.0  0.0  0.0  142.833333  
2024-01-19 22:00:00+00:00      30.441667  0.0  0.0  0.0  148.916667  
2024-01-19 23:00:00+00:00      28.975000  0.0  0.0  0.0  151.000000  

[35520 rows x 9 columns]

# Wettervorhersage

In [95]:
# read the data from the csv file
forecast_data = pd.read_csv('../0_data/forecasts/forecasts_22_12_2023.csv', index_col='period_end', parse_dates=True, usecols=['ghi','dni','dhi','air_temp','zenith','azimuth','cloud_opacity','period_end'])
# format datime index
print(forecast_data.index.dtype)
# datentyp index in umwandeln
forecast_data.index = pd.to_datetime(forecast_data.index, format='ISO8601')
# utc time erstellen für index länge erstellen
time_index = pd.date_range(start=forecast_data.index[0], periods=len(forecast_data), freq='30min')
forecast_data.index = time_index

# period end zu perod begin verschieben
forecast_data.index = forecast_data.index.shift(-1, freq='30min')
# resample auf 60min
forecast_data = forecast_data.resample('60min').mean()

forecast_data 

object


ghi   dni    dhi  air_temp  zenith  azimuth  \
2023-12-22 15:00:00+00:00    1.5   0.0    1.5       6.0    92.0    125.0   
2023-12-22 16:00:00+00:00    0.0   0.0    0.0       6.0   100.5    114.0   
2023-12-22 17:00:00+00:00    0.0   0.0    0.0       6.0   109.5    103.5   
2023-12-22 18:00:00+00:00    0.0   0.0    0.0       6.0   119.5     93.0   
2023-12-22 19:00:00+00:00    0.0   0.0    0.0       6.0   129.0     81.0   
...                          ...   ...    ...       ...     ...      ...   
2023-12-29 11:00:00+00:00  168.5  78.5  145.0       6.0    72.5     -2.5   
2023-12-29 12:00:00+00:00  153.5  81.5  131.0       6.0    74.0    163.5   
2023-12-29 13:00:00+00:00  105.0  30.0   98.0       6.0    78.0    149.5   
2023-12-29 14:00:00+00:00   43.0   0.0   43.0       6.0    84.0    137.0   
2023-12-29 15:00:00+00:00    5.0   0.0    5.0       6.0    89.0    128.0   

                           cloud_opacity  
2023-12-22 15:00:00+00:00           53.0  
2023-12-22 16:00:00+00:00           63.0  
2023-12-22 17:00:00+00:00           54.5  
2023-12-22 18:00:00+00:00           64.0  
2023-12-22 19:00:00+00:00           66.0  
...                                  ...  
2023-12-29 11:00:00+00:00           40.0  
2023-12-29 12:00:00+00:00           38.5  
2023-12-29 13:00:00+00:00           41.0  
2023-12-29 14:00:00+00:00           43.5  
2023-12-29 15:00:00+00:00           43.0  

[169 rows x 7 columns]

# Daten zusammenführen und auf gleiche Spalten wie im Training zurückführen

In [96]:
# Zeiträume ausschneiden
data_fitting_interval = historic_data['2020-01-01 09:00:00':'2023-01-23 17:59:59'] #24 Stunden = Window + 27 Stunden für Lag Features verSchiebdung diese werden beim verschieben nan und entfallen

historic_data_test_interval = historic_data['2023-12-22 00:00:00':'2023-12-22 15:59:59'] #24 Stunden = Window + 27 Stunden für Lag Features verSchiebdung diese werden beim verschieben nan und entfallen
pv_power_test_interval = historic_data[['pv_power1','pv_power2']]['2023-12-22 16:00:00':'2023-12-27 20:59:59']
forecast_data_test_interval = forecast_data['2023-12-22 16:00:00':'2023-12-27 20:59:59'] #24 Stunden + 27 Stunden für Lag Features verSchiebdung diese werden beim verschieben nan und entfallen


# Spalten hinzufügen
forecast_data_test_interval['pv_power1'] = pv_power_test_interval['pv_power1']
forecast_data_test_interval['pv_power2'] = pv_power_test_interval['pv_power2']

print(type(forecast_data_test_interval))

# forecast data an historic data anhängen
merged_df = pd.concat([historic_data_test_interval, forecast_data_test_interval])
#merged_df = historic_data_test_interval


'''# Trainierte Namen ['zenith', 'AirTemp', 'azimuth_solcast', 'CloudOpacity', 'Dni', 'Ebh', 'Ghi', 'Dhi', 'pv_power1', 'pv_power2']
# pv_power1	pv_power2	air_temp	azimuth	cloud_opacity	dhi	dni	ghi	zenith
# Spalten umbenennen
merged_df.rename(columns={'air_temp':'AirTemp', 
                          'azimuth':'azimuth_solcast', 
                          'cloud_opacity':'CloudOpacity',
                          'dni':'Dni', 
                          'dhi':'Dhi', 
                          'ghi':'Ghi'}, inplace=True)


# Spalten sortieren nach # zenith	AirTemp	azimuth_solcast	CloudOpacity	Dni	Ebh	Ghi	Dhi	pv_power1	pv_power2' 
merged_df = merged_df[['zenith', 'AirTemp', 'azimuth_solcast', 'CloudOpacity', 'Dni', 'Ghi', 'Dhi', 'pv_power1', 'pv_power2']]'''

merged_df
# ['pv_power1', 'pv_power2', 'air_temp', 'azimuth', 'cloud_opacity', 'dhi', 'dni', 'ghi', 'zenith']
#plot.timeseries(merged_df)

<class 'pandas.core.frame.DataFrame'>


pv_power1  pv_power2  air_temp     azimuth  \
2023-12-22 00:00:00+00:00 -9.629651e-35        0.0       3.0  -35.583333   
2023-12-22 01:00:00+00:00 -9.629651e-35        0.0       3.0  -57.250000   
2023-12-22 02:00:00+00:00 -9.629651e-35        0.0       3.0  -73.500000   
2023-12-22 03:00:00+00:00 -9.629651e-35        0.0       3.5  -86.500000   
2023-12-22 04:00:00+00:00 -9.629651e-35        0.0       3.5  -97.500000   
...                                 ...        ...       ...         ...   
2023-12-27 16:00:00+00:00  0.000000e+00        0.0       7.0  114.500000   
2023-12-27 17:00:00+00:00  0.000000e+00        0.0       7.0  104.000000   
2023-12-27 18:00:00+00:00  0.000000e+00        0.0       6.0   93.000000   
2023-12-27 19:00:00+00:00  0.000000e+00        0.0       6.0   81.500000   
2023-12-27 20:00:00+00:00  0.000000e+00        0.0       6.0   67.500000   

                           cloud_opacity  dhi  dni  ghi      zenith  
2023-12-22 00:00:00+00:00      61.408333  0.0  0.0  0.0  150.416667  
2023-12-22 01:00:00+00:00      54.250000  0.0  0.0  0.0  143.250000  
2023-12-22 02:00:00+00:00      56.125000  0.0  0.0  0.0  134.416667  
2023-12-22 03:00:00+00:00      67.950000  0.0  0.0  0.0  124.750000  
2023-12-22 04:00:00+00:00      66.800000  0.0  0.0  0.0  115.000000  
...                                  ...  ...  ...  ...         ...  
2023-12-27 16:00:00+00:00      58.000000  0.0  0.0  0.0  100.000000  
2023-12-27 17:00:00+00:00      58.000000  0.0  0.0  0.0  109.500000  
2023-12-27 18:00:00+00:00      58.000000  0.0  0.0  0.0  118.500000  
2023-12-27 19:00:00+00:00      56.500000  0.0  0.0  0.0  128.500000  
2023-12-27 20:00:00+00:00      54.500000  0.0  0.0  0.0  138.000000  

[141 rows x 9 columns]

# LSTM Klasse

In [97]:
#TODO auslagern
########################################################################################################################

# LSTM-Modell definieren
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        # Anzahl der neuronale Netze in der LSTM-Schicht
        self.hidden_size = hidden_size
        # Anzahl der LSTM-Schichten
        self.num_layers = num_layers
        # LSTM-Schicht
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        # Die Ausgabe der LSTM-Schicht wird an eine lineare Schicht weitergegeben
        self.fc = nn.Linear(hidden_size, output_size)
        self.hidden = None

    def forward(self, x):
        # hidden state durch reichen führt zu fehler ?
        if 1:# self.hidden == None:
            # multiplizieren der Anzahl der LSTM-Schichten mit 2, da bidirektional
            # Initialisieren der versteckten Zustände, dieser beinhaltet die Information über die vorherigen Zustände
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device).requires_grad_()
            # Cell state initialisieren, dieser beinhaltet die Information über die vorherigen Zustände
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device).requires_grad_()
            hn, cn = h0, c0
        else:
            hn, cn = self.hidden

        # Forward Propagation, Berechnung des LSTM layers
        out, hidden = self.lstm(x, (hn, cn))
        

        # Ausgabe in der linearen Schicht berechnen
        out = self.fc(out[:, -1, :])
        # (batch_size, sequence_length)
        self.hidden = hidden
        return out


########################################################################################################################

# Daten vorbereiten

In [98]:
### Features erweitern mit Zeitverschiebungen
def shift_and_merge(df, n_shifts):
    full_data = df.copy()
    
    # Verschieben der Daten in die Zukunft und in die Vergangenheit
    for i in range(1, n_shifts + 1):
        shifted_future = df.shift(periods=-i)
        shifted_past = df.shift(periods=i)
        
        # Spalten umbenennen
        shifted_future.columns = [f"{column}_n+{i}" for column in shifted_future.columns]
        shifted_past.columns = [f"{column}_n-{i}" for column in shifted_past.columns]
        
        # verschobene Daten hinzufügen
        full_data[shifted_future.columns] = shifted_future
        full_data[shifted_past.columns] = shifted_past
    
    return full_data

time_shift = 27
full_data = shift_and_merge(merged_df, time_shift)
full_data

pv_power1  pv_power2  air_temp     azimuth  \
2023-12-22 00:00:00+00:00 -9.629651e-35        0.0       3.0  -35.583333   
2023-12-22 01:00:00+00:00 -9.629651e-35        0.0       3.0  -57.250000   
2023-12-22 02:00:00+00:00 -9.629651e-35        0.0       3.0  -73.500000   
2023-12-22 03:00:00+00:00 -9.629651e-35        0.0       3.5  -86.500000   
2023-12-22 04:00:00+00:00 -9.629651e-35        0.0       3.5  -97.500000   
...                                 ...        ...       ...         ...   
2023-12-27 16:00:00+00:00  0.000000e+00        0.0       7.0  114.500000   
2023-12-27 17:00:00+00:00  0.000000e+00        0.0       7.0  104.000000   
2023-12-27 18:00:00+00:00  0.000000e+00        0.0       6.0   93.000000   
2023-12-27 19:00:00+00:00  0.000000e+00        0.0       6.0   81.500000   
2023-12-27 20:00:00+00:00  0.000000e+00        0.0       6.0   67.500000   

                           cloud_opacity  dhi  dni  ghi      zenith  \
2023-12-22 00:00:00+00:00      61.408333  0.0  0.0  0.0  150.416667   
2023-12-22 01:00:00+00:00      54.250000  0.0  0.0  0.0  143.250000   
2023-12-22 02:00:00+00:00      56.125000  0.0  0.0  0.0  134.416667   
2023-12-22 03:00:00+00:00      67.950000  0.0  0.0  0.0  124.750000   
2023-12-22 04:00:00+00:00      66.800000  0.0  0.0  0.0  115.000000   
...                                  ...  ...  ...  ...         ...   
2023-12-27 16:00:00+00:00      58.000000  0.0  0.0  0.0  100.000000   
2023-12-27 17:00:00+00:00      58.000000  0.0  0.0  0.0  109.500000   
2023-12-27 18:00:00+00:00      58.000000  0.0  0.0  0.0  118.500000   
2023-12-27 19:00:00+00:00      56.500000  0.0  0.0  0.0  128.500000   
2023-12-27 20:00:00+00:00      54.500000  0.0  0.0  0.0  138.000000   

                           pv_power1_n+1  ...  zenith_n+27  pv_power1_n-27  \
2023-12-22 00:00:00+00:00  -9.629651e-35  ...        124.5             NaN   
2023-12-22 01:00:00+00:00  -9.629651e-35  ...        115.5             NaN   
2023-12-22 02:00:00+00:00  -9.629651e-35  ...        105.5             NaN   
2023-12-22 03:00:00+00:00  -9.629651e-35  ...         96.5             NaN   
2023-12-22 04:00:00+00:00  -9.629651e-35  ...         88.5             NaN   
...                                  ...  ...          ...             ...   
2023-12-27 16:00:00+00:00   0.000000e+00  ...          NaN    4.068333e+01   
2023-12-27 17:00:00+00:00   0.000000e+00  ...          NaN    1.858333e+01   
2023-12-27 18:00:00+00:00   0.000000e+00  ...          NaN    0.000000e+00   
2023-12-27 19:00:00+00:00   0.000000e+00  ...          NaN    0.000000e+00   
2023-12-27 20:00:00+00:00            NaN  ...          NaN   -5.777791e-35   

                           pv_power2_n-27  air_temp_n-27  azimuth_n-27  \
2023-12-22 00:00:00+00:00             NaN            NaN           NaN   
2023-12-22 01:00:00+00:00             NaN            NaN           NaN   
2023-12-22 02:00:00+00:00             NaN            NaN           NaN   
2023-12-22 03:00:00+00:00             NaN            NaN           NaN   
2023-12-22 04:00:00+00:00             NaN            NaN           NaN   
...                                   ...            ...           ...   
2023-12-27 16:00:00+00:00    4.327833e+02            6.5         149.5   
2023-12-27 17:00:00+00:00    1.175000e+01            6.0         137.0   
2023-12-27 18:00:00+00:00    0.000000e+00            5.5         125.0   
2023-12-27 19:00:00+00:00    0.000000e+00            5.0         114.5   
2023-12-27 20:00:00+00:00   -5.777791e-35            5.0         103.5   

                           cloud_opacity_n-27  dhi_n-27  dni_n-27  ghi_n-27  \
2023-12-22 00:00:00+00:00                 NaN       NaN       NaN       NaN   
2023-12-22 01:00:00+00:00                 NaN       NaN       NaN       NaN   
2023-12-22 02:00:00+00:00                 NaN       NaN       NaN       NaN   
2023-12-22 03:00:00+00:00                 NaN       NaN       NaN       NaN   
2023-12-22 04:00:00+00:00                 N

In [99]:
# Funktion zum Generieren der Feature-Namen
def generate_feature_names(base_features, n_values, future=True, historic=False):
    feature_names = base_features.copy()
    for n in range(1, n_values + 1):
        for feature in base_features:
            if future and feature != 'pv_power1' and feature != 'pv_power2':
                feature_names.append(f'{feature}_n+{n}')
            if historic:
                # raus da er es theoretisch über das lstm window lernen kann
                pass
                #feature_names.append(f'{feature}_n-{n}')
            if feature != 'pv_power1' and feature != 'pv_power2':
                pass
                #feature_names.append(f'{feature}')
    return feature_names

# Definieren der Basis-Features
base_features = ['pv_power1', 'pv_power2', 'air_temp', 'azimuth', 'cloud_opacity', 'dhi', 'dni', 'ghi', 'zenith']

# Generieren der Feature-Namen
future_and_historic_values = 27 # beachte historische Werte werden nicht verwendet
future_and_historic_features = generate_feature_names(base_features, future_and_historic_values, future=True, historic=True)

# Festlegen der Zielvariable für die Vorhersage
# Ziel Variable ist der +n wert von pv_power2
prediction_target = 'pv_power2_n+1'

# Namen der Spalten, in denen NaN-Werte und Null-Werte überprüft werden sollen
columns_to_check_nan = future_and_historic_features.copy()
columns_to_check_nan.append(prediction_target)


# Eigene Klasse für die Pipeline zum Entfernen von Zeilen mit NaN-Werten,  Null-Werten und negativen Werten
class DropRowsWithNaNAndZero(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_check_nan=None, columns_to_check_zero=None, columns_to_check_negative=None):
        self.columns_to_check_nan = columns_to_check_nan
        self.columns_to_check_zero = columns_to_check_zero
        self.columns_to_check_negative = columns_to_check_negative

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Kopie des DataFrames erstellen, um das Original nicht zu ändern
        X_copy = X.copy()

        if self.columns_to_check_nan is not None:
            # Zeilen mit NaN-Werten in den angegebenen Spalten entfernen
            X_copy.dropna(subset=self.columns_to_check_nan, inplace=True)
        
        if self.columns_to_check_zero is not None:
            # Zeilen mit Null-Werten in den angegebenen Spalten entfernen
            for col in self.columns_to_check_zero:
                pass
                #X_copy = X_copy[X_copy[col] != 0]
                
        if self.columns_to_check_negative is not None:
            # Zeilen mit positen-Werten in den angegebenen Spalten markieren 
            # dadurch werden negative entfernt
            for col in self.columns_to_check_negative:
                X_copy[X_copy[col] < 0] = 0
            pass

        return X_copy

# Namen der Spalten, in denen NaN-Werte und Null-Werte überprüft werden sollen
columns_to_check_zero = []#['pv_power2', 'pv_power1']
columns_to_check_negative = []
for featere in future_and_historic_features:
    if 'pv_power' in featere:
        columns_to_check_negative.append(featere)

# 1. Pipeline mit Dataframes
# Erstellen der Pipeline
pre_processor = Pipeline([
    ('data_cleaning', DropRowsWithNaNAndZero(columns_to_check_nan, columns_to_check_zero, columns_to_check_negative)),
])
full_data = pre_processor.transform(full_data)


# Erstellen verschiedener feature sets
feature_set0 = future_and_historic_features


print(feature_set0)

# Daten übergeben
data_set = full_data

# Auswählen der Zielvariablen und der benutzten Features
# Spalten für die Vorhersage und die Features
x_columns = future_and_historic_features
y_columns = ['pv_power2_n+1']  # prediction column aus modell eintragen



#display(data_set)
#plot.timeseries(data_set)


['pv_power1', 'pv_power2', 'air_temp', 'azimuth', 'cloud_opacity', 'dhi', 'dni', 'ghi', 'zenith', 'air_temp_n+1', 'azimuth_n+1', 'cloud_opacity_n+1', 'dhi_n+1', 'dni_n+1', 'ghi_n+1', 'zenith_n+1', 'air_temp_n+2', 'azimuth_n+2', 'cloud_opacity_n+2', 'dhi_n+2', 'dni_n+2', 'ghi_n+2', 'zenith_n+2', 'air_temp_n+3', 'azimuth_n+3', 'cloud_opacity_n+3', 'dhi_n+3', 'dni_n+3', 'ghi_n+3', 'zenith_n+3', 'air_temp_n+4', 'azimuth_n+4', 'cloud_opacity_n+4', 'dhi_n+4', 'dni_n+4', 'ghi_n+4', 'zenith_n+4', 'air_temp_n+5', 'azimuth_n+5', 'cloud_opacity_n+5', 'dhi_n+5', 'dni_n+5', 'ghi_n+5', 'zenith_n+5', 'air_temp_n+6', 'azimuth_n+6', 'cloud_opacity_n+6', 'dhi_n+6', 'dni_n+6', 'ghi_n+6', 'zenith_n+6', 'air_temp_n+7', 'azimuth_n+7', 'cloud_opacity_n+7', 'dhi_n+7', 'dni_n+7', 'ghi_n+7', 'zenith_n+7', 'air_temp_n+8', 'azimuth_n+8', 'cloud_opacity_n+8', 'dhi_n+8', 'dni_n+8', 'ghi_n+8', 'zenith_n+8', 'air_temp_n+9', 'azimuth_n+9', 'cloud_opacity_n+9', 'dhi_n+9', 'dni_n+9', 'ghi_n+9', 'zenith_n+9', 'air_temp_n

In [100]:
# Übergabe der Hyperparameter
hidden_size = 150 # Anzahl der Neuronen in der LSTM-Schicht
num_layers = 2 #  Anzahl der LSTM-Schichten

# z.B. bis n+3 vorhersagen
horrizon = 24 # Eintragen der Parameter des Modells
# n-6 zur Vorhersage verwenden
window = 48 # Eintragen der Parameter des Modells

# horizont( z.B von n-6 bis n-1) + window (z.B. n+1 bis n+3) + 1 (n) = sequence_length
sequence_length = horrizon + window +1 # Länge einer Sequenz
#sequence_length = params['sequence_length']  # Länge einer gesamten Sequenz

# Input und Output Größe des Modells festlegen
input_size = len(x_columns) # Alle Spalten außer den prognostizierten Wert


# Daten in Sequenzen aufteilen
sequences = []

# Anpassen an urpsrüngliche Trainingsdaten
data_fitting = shift_and_merge(data_fitting_interval, time_shift)
data_fitting = pre_processor.transform(data_fitting)
#data_fitting 
scaler_x = MinMaxScaler(feature_range=(0, 1)).fit(data_fitting[x_columns])
scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(data_fitting[y_columns])

# Die Schleife teilt die Daten in Sequenzen auf
# eine Sequenz besteht aus z.B. den 24 vorherigen Werten und dem nächsten Wert
# die Sprünge sind jeweils von z.B.:
# x:1 bis 24 prognositierziert y:z.b. 25-28
# x:2 bis 25 prognostiziert y:26-29
# x:3 bis 26 prognostiziert y:27-30
# ... 
print(len(data_set))
# Range Maximum berechnen: durch test am ende ermittelt differnz von 2 bei horrizon von 3 festgestellt
for i in range(len(data_set) - (sequence_length)):
    X = data_set[x_columns].iloc[i:i + window + 1, :] # i - i + sequence_length sind die Werte die für die Vorhersage verwendet werden
    X = scaler_x.transform(X)  # Daten skalieren
    y = data_set[y_columns].iloc[i + window+1:i + window+horrizon+1]  # Annahme: Die Ausgabe ist die erste Spalte
    y = scaler_y.transform(y)  
    sequences.append((X, y))
    
    
display(len(sequences))


114


41

# Deployment

In [101]:
test_data = sequences

# Anzahl der Zielvariablen = Anzahl der Ausgabeneuronen = Horrizon
output_size = horrizon



########################################################################################################################
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Laden des Modells
model = torch.load('multi rmse_1170, Learningrate_0.0001, Horrizon_24, Window_48, Layer_2, epoch_99, hidden_150, feature_set0, batch_8, pc_pv_power2_n+1, fh_27, ts_27.pt', map_location=torch.device('cpu'))


# Test des Modells
# Modell in den Evaluationsmodus versetzen
model.eval()
real_values = []
predictions = []
# Vorhersage ohne Berechnung der Gradienten
with torch.no_grad():
    # Schleife mit höherer Schrittegröße, sodass y-Werte für die Vorhersage nicht überlappen
    # also prediction y+1, y+2, y+3 dann drei schritte weiter
    for i in range(0, len(test_data), horrizon):
        X, y = test_data[i]
        # Eine Sequenz aus den Testdaten auswählen
        # unsqueeze(0) fügt eine Dimension hinzu, da das Modell immer eine Batch-Größe erwartet
        # to(device) lädt die Daten auf das Gerät
        # LSTM-Modell erwartet eine Eingabe mit der Form (batch_size, sequence_length, input_size)
        test_input = torch.from_numpy(X).float().unsqueeze(0).to(device)
        # Einen Wert vorhersagen
        prediction = model(test_input).cpu().numpy().flatten()
        prediction = scaler_y.inverse_transform(prediction.reshape(-1, 1))
        y = scaler_y.inverse_transform(y.reshape(-1, 1))
        
        for pred_value, real_value in zip(prediction, y):
            if pred_value[0] < 0:
                pred_value[0] = 0
            predictions.append(pred_value[0])
            real_values.append(real_value[0])
        
display(type(prediction)) 
# die letzten Predcitions und real_values entfernen, da diese nicht mehr vollständig sind
predictions = predictions[:len(data_set.index[0:len(sequences)-1])]
real_values = real_values[:len(data_set.index[0:len(sequences)-1])]


result = pd.DataFrame()
result['real_values'] = real_values
result['predictions'] = predictions
(result)


# Index des Trainingdatensatzes für die Visualisierung
print(len(result.index))
print(len(data_set.index[0:len(sequences)-1]))
result.index = data_set.index[0:len(sequences)-1]
rmse = mean_squared_error(result['real_values'], result['predictions'], squared=False)

print(rmse)
plot.timeseries(result, stacked=False, save_on_disk=True, plot_label=f'Deployment MultitimeHorizon LSTM - RMSE {rmse:.1f}')

numpy.ndarray

40
40
668.6101851001405
